In [25]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
import json 
import matplotlib
import matplotlib.pyplot as plt


from sklearn.metrics import jaccard_score
from scipy.spatial.distance import directed_hausdorff

import cv2
import tifffile
import seaborn as sns
from itertools import chain
from PIL import Image
from PIL import ImageDraw

import skimage
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import scipy.stats as stats

In [ ]:
#Read Metrics file for the team
BASE_PATH = os.getcwd()+'/'
INPUT_PATH = BASE_PATH
print(INPUT_PATH)

team = "team_1"
df_metrics_team1 = pd.read_csv(INPUT_PATH+f'winning-submissions/{team}/metrics.csv')

team = "team_2"
df_metrics_team2 = pd.read_csv(INPUT_PATH+f'winning-submissions/{team}/metrics.csv')

team = "team_3"
df_metrics_team3 = pd.read_csv(INPUT_PATH+f'winning-submissions/{team}/metrics.csv')


In [3]:
df_metrics_team1.head(3)

,id,dice_score,iou_score,hd_score,hd95_score,filename,tissue_name
0,3417,0.818191,0.692320,157.581725,155.066639,D116-RML-BPS-2-2_0_0_22_lung,lung
1,9077,0.651993,0.483672,180.693110,246.861231,D071-RLL-17B5-2_1323_1323_27_lung,lung
2,474,0.704360,0.543639,517.000967,499.022386,D356-RLL-12A3-2_0_0_16_lung,lung


In [4]:
print(df_metrics_team1["dice_score"].mean())
print(df_metrics_team2["dice_score"].mean())
print(df_metrics_team3["dice_score"].mean())

0.8356159833475378
0.8339292939248892
0.8326609412139847


In [5]:
print(df_metrics_team1["iou_score"].mean())
print(df_metrics_team2["iou_score"].mean())
print(df_metrics_team3["iou_score"].mean())

0.7384615995430411
0.7362468557228992
0.7333478375569404


In [6]:
print(df_metrics_team1["hd_score"].mean())
print(df_metrics_team2["hd_score"].mean())
print(df_metrics_team3["hd_score"].mean())

201.99175323853962
261.016577773816
268.1459310778021


In [7]:
print(df_metrics_team1["hd95_score"].mean())
print(df_metrics_team2["hd95_score"].mean())
print(df_metrics_team3["hd95_score"].mean())

380.92824735591137
392.336002327034
406.3626494362451


In [8]:
df_metrics_team1["team"] = "Team 1"
df_metrics_team2["team"] = "Team 2"
df_metrics_team3["team"] = "Team 3"
df_metrics_team3.head()

,id,dice_score,iou_score,hd_score,hd95_score,filename,tissue_name,team
0,3417,0.796006,0.661138,311.852529,299.903047,D116-RML-BPS-2-2_0_0_22_lung,lung,Team 3
1,9077,0.757720,0.609943,157.543645,178.041250,D071-RLL-17B5-2_1323_1323_27_lung,lung,Team 3
2,474,0.655108,0.487108,210.800854,208.958512,D356-RLL-12A3-2_0_0_16_lung,lung,Team 3
3,22423,0.793068,0.657094,606.469290,583.745332,D231-RLL-13B4-2_0_0_32_lung,lung,Team 3
4,16724,0.594874,0.423360,450.165525,430.694517,D239-RLL-23C4-2_0_0_45_lung,lung,Team 3


In [49]:
def print_stats(df_metrics_team1, df_metrics_team2, df_metrics_team3, metric, case, per_organ):
    if case == "best":
        if per_organ is True:
            indices_to_drop = df_metrics_team1.groupby('tissue_name')[metric].nlargest(1).index.get_level_values(1)
            team1_stats = df_metrics_team1.drop(indices_to_drop, inplace=False)

            indices_to_drop = df_metrics_team2.groupby('tissue_name')[metric].nlargest(1).index.get_level_values(1)
            team2_stats = df_metrics_team2.drop(indices_to_drop, inplace=False)

            indices_to_drop = df_metrics_team3.groupby('tissue_name')[metric].nlargest(1).index.get_level_values(1)
            team3_stats = df_metrics_team3.drop(indices_to_drop, inplace=False)
        
        elif per_organ is False:
            df_metrics_team1_sorted = df_metrics_team1.sort_values(metric, ascending=False)
            df_metrics_team1_sorted = df_metrics_team1_sorted.reset_index(drop=True)
            team1_stats = df_metrics_team1_sorted.drop(df_metrics_team1_sorted[df_metrics_team1_sorted['id'] == df_metrics_team1_sorted.id[0]].index)

            df_metrics_team2_sorted = df_metrics_team2.sort_values(metric, ascending=False)
            df_metrics_team2_sorted = df_metrics_team2_sorted.reset_index(drop=True)
            team2_stats = df_metrics_team2_sorted.drop(df_metrics_team2_sorted[df_metrics_team2_sorted['id'] == df_metrics_team2_sorted.id[0]].index)

            df_metrics_team3_sorted = df_metrics_team3.sort_values(metric, ascending=False)
            df_metrics_team3_sorted = df_metrics_team3_sorted.reset_index(drop=True)
            team3_stats = df_metrics_team3_sorted.drop(df_metrics_team3_sorted[df_metrics_team3_sorted['id'] == df_metrics_team3_sorted.id[0]].index)

    elif case == "worst":
        if per_organ is True:
            indices_to_drop = df_metrics_team1.groupby('tissue_name')[metric].nsmallest(1).index.get_level_values(1)
            team1_stats = df_metrics_team1.drop(indices_to_drop, inplace=False)

            indices_to_drop = df_metrics_team2.groupby('tissue_name')[metric].nsmallest(1).index.get_level_values(1)
            team2_stats = df_metrics_team2.drop(indices_to_drop, inplace=False)

            indices_to_drop = df_metrics_team3.groupby('tissue_name')[metric].nsmallest(1).index.get_level_values(1)
            team3_stats = df_metrics_team3.drop(indices_to_drop, inplace=False)
        
        elif per_organ is False:
            df_metrics_team1_sorted = df_metrics_team1.sort_values(metric, ascending=True)
            df_metrics_team1_sorted = df_metrics_team1_sorted.reset_index(drop=True)
            team1_stats = df_metrics_team1_sorted.drop(df_metrics_team1_sorted[df_metrics_team1_sorted['id'] == df_metrics_team1_sorted.id[0]].index)
            
            df_metrics_team2_sorted = df_metrics_team2.sort_values(metric, ascending=True)
            df_metrics_team2_sorted = df_metrics_team2_sorted.reset_index(drop=True)
            team2_stats = df_metrics_team2_sorted.drop(df_metrics_team2_sorted[df_metrics_team2_sorted['id'] == df_metrics_team2_sorted.id[0]].index)

            df_metrics_team3_sorted = df_metrics_team3.sort_values(metric, ascending=True)
            df_metrics_team3_sorted = df_metrics_team3_sorted.reset_index(drop=True)
            team3_stats = df_metrics_team3_sorted.drop(df_metrics_team3_sorted[df_metrics_team3_sorted['id'] == df_metrics_team3_sorted.id[0]].index)

    print("Team 1: ", team1_stats[metric].mean())
    print("Team 2: ", team2_stats[metric].mean())
    print("Team 3: ", team3_stats[metric].mean())

In [10]:
# Case: Remove worst 1
metric = "dice_score"
case = "worst"
per_organ = False
print_stats(df_metrics_team1, df_metrics_team2, df_metrics_team3, metric, case, per_organ)


Team 1:  0.8390049417488906
Team 2:  0.8371305822731747
Team 3:  0.8359230797012543


In [11]:
# Case: Remove top 1
metric = "dice_score"
case = "best"
per_organ = False
print_stats(df_metrics_team1, df_metrics_team2, df_metrics_team3, metric, case, per_organ)


Team 1:  0.8349805926277769
Team 2:  0.8332769934647581
Team 3:  0.8320261150047805


In [32]:
# Case: Remove worst 1 per organ (total 5)
metric = "dice_score"
case = "worst"
per_organ = True
print_stats(df_metrics_team1, df_metrics_team2, df_metrics_team3, metric, case, per_organ)

Team 1:  0.8421404631583568
Team 2:  0.8418451263026939
Team 3:  0.8413001279427224


In [33]:
# Case: Remove worst 1 per organ (total 5)
metric = "iou_score"
case = "worst"
per_organ = True
print_stats(df_metrics_team1, df_metrics_team2, df_metrics_team3, metric, case, per_organ)

Team 1:  0.7452622222838429
Team 2:  0.7446923328236542
Team 3:  0.7423267174791883


In [13]:
# Case: Remove best 1 per organ (total 5)
metric = "dice_score"
case = "best"
per_organ = True
print_stats(df_metrics_team1, df_metrics_team2, df_metrics_team3, metric, case, per_organ)

Team 1:  0.8332129926957113
Team 2:  0.8314839180250487
Team 3:  0.8301547648991853


In [14]:
# Case: Remove worst 1 per organ (total 5), Use case="best" since for HD, lower is better.
metric = "hd95_score"
case = "best"
per_organ = True
print_stats(df_metrics_team1, df_metrics_team2, df_metrics_team3, metric, case, per_organ)

Team 1:  365.716080585105
Team 2:  376.51077072259056
Team 3:  390.4159526441003


In [15]:
# Case: Remove best 1 per organ (total 5), Use case="best" since for HD, lower is better.
metric = "hd95_score"
case = "worst"
per_organ = True
print_stats(df_metrics_team1, df_metrics_team2, df_metrics_team3, metric, case, per_organ)

Team 1:  387.5384541536713
Team 2:  399.0265691585663
Team 3:  413.1090463732476


In [47]:
# Case: Remove worst 3 per organ (total 15) [need to change n to 3 in print_stats function for this]
metric = "dice_score"
case = "worst"
per_organ = True
print_stats(df_metrics_team1, df_metrics_team2, df_metrics_team3, metric, case, per_organ)

Team 1:  0.8505948863771244
Team 2:  0.8503647579377415
Team 3:  0.8508701207444114


In [48]:
# Case: Remove worst 3 per organ (total 15) [need to change n to 3 in print_stats function for this]
metric = "iou_score"
case = "worst"
per_organ = True
print_stats(df_metrics_team1, df_metrics_team2, df_metrics_team3, metric, case, per_organ)

Team 1:  0.7554817330409723
Team 2:  0.7548293042908264
Team 3:  0.7538377446873296


In [40]:
metric="dice_score"
indices_to_drop = df_metrics_team1[metric].nsmallest(5).index.get_level_values(0)
team1_stats = df_metrics_team1.drop(indices_to_drop, inplace=False)
team1_stats[metric].mean()

0.8463438907519583

In [41]:
metric="dice_score"
indices_to_drop = df_metrics_team2[metric].nsmallest(5).index.get_level_values(0)
team2_stats = df_metrics_team2.drop(indices_to_drop, inplace=False)
team2_stats[metric].mean()

0.8452835771231538

In [42]:
metric="dice_score"
indices_to_drop = df_metrics_team3[metric].nsmallest(5).index.get_level_values(0)
team3_stats = df_metrics_team3.drop(indices_to_drop, inplace=False)
team3_stats[metric].mean()

0.844147576376886

In [43]:
metric="iou_score"
indices_to_drop = df_metrics_team1[metric].nsmallest(5).index.get_level_values(0)
team1_stats = df_metrics_team1.drop(indices_to_drop, inplace=False)
team1_stats[metric].mean()

0.7497321521745767

In [44]:
metric="iou_score"
indices_to_drop = df_metrics_team2[metric].nsmallest(5).index.get_level_values(0)
team2_stats = df_metrics_team2.drop(indices_to_drop, inplace=False)
team2_stats[metric].mean()

0.7480103939957221

In [45]:
metric="iou_score"
indices_to_drop = df_metrics_team3[metric].nsmallest(5).index.get_level_values(0)
team3_stats = df_metrics_team3.drop(indices_to_drop, inplace=False)
team3_stats[metric].mean()

0.7451230813785811

# Kendall's Tau between public and private leaderboard

In [27]:

df_all_teams = pd.read_csv(INPUT_PATH+f'winning-submissions/all-team-scores.csv')
df_all_teams.head(5)

,DateSubmitted,SubmissionId,TeamId,UserName,TeamName,IsSelected,PublicScore,PrivateScore
0,2022-06-23 02:15:53,27177301,8805846,grapestone5321,Naruhiko Nakanishi,False,0.000178,0.000168
1,2022-06-23 02:44:43,27177527,8806966,nuoyanchen,Alvin Chen,False,0.000178,0.000168
2,2022-06-23 03:34:59,27177973,8806540,tanakar,tk,False,0.195355,0.193622
3,2022-06-23 04:12:12,27178221,8806540,tanakar,tk,False,0.462739,0.344833
4,2022-06-23 08:32:20,27180469,8806540,tanakar,tk,False,0.567092,0.467679


In [28]:
df_all_teams.SubmissionId.count()

35402

In [29]:
len(df_all_teams.TeamId.unique())

1174

In [30]:
df_all_teams_submission = df_all_teams[df_all_teams["IsSelected"] == True]
df_all_teams_submission.head()

,DateSubmitted,SubmissionId,TeamId,UserName,TeamName,IsSelected,PublicScore,PrivateScore
10,2022-06-24 00:34:14,27188747,8805846,grapestone5321,Naruhiko Nakanishi,True,0.000178,0.000168
22,2022-06-24 07:31:25,27191942,8805826,kaerunantoka,kaerururu,True,0.491458,0.414213
201,2022-06-27 07:26:20,27226232,8814397,arnavr10880,ArnavR,True,0.644654,0.614323
244,2022-06-27 18:44:19,27232444,8818117,susnato,Susnato Dhar,True,0.535654,0.469059
285,2022-06-28 06:58:31,27237116,8807562,alexandergod,alex,True,0.644654,0.614323


In [31]:
df_all_teams_submission.SubmissionId.count()

647

In [32]:
df_all_teams_submission.TeamId.count()

647

In [33]:
len(df_all_teams_submission.TeamId.unique())

365

In [ ]:
# ranking based on private vs ranking based on public
df_all_teams_sorted_private = df_all_teams.sort_values("PrivateScore", ascending=False)
df_all_teams_sorted_private.reset_index(drop=True, inplace=True)

df_all_teams_sorted_public = df_all_teams.sort_values("PublicScore", ascending=False)
df_all_teams_sorted_public.reset_index(drop=True, inplace=True)

public_lb = df_all_teams_sorted_public['TeamName']

private_lb = df_all_teams_sorted_private['TeamName']

tau, p_value = stats.kendalltau(public_lb, private_lb)

# Print the calculated tau and p-value
print("Kendall's tau:", tau)
print("p-value:", p_value)

In [68]:
public_lb = df_all_teams['PublicScore']

private_lb = df_all_teams['PrivateScore']

tau, p_value = stats.kendalltau(public_lb, private_lb)

# Print the calculated tau and p-value
print("Kendall's tau:", tau)
print("p-value:", p_value)

Kendall's tau: 0.7526500663696196
p-value: 0.0


In [69]:
public_lb = df_all_teams_submission['PublicScore']
private_lb = df_all_teams_submission['PrivateScore']
tau, p_value = stats.kendalltau(public_lb, private_lb)

# Print the calculated tau and p-value
print("Kendall's tau:", tau)
print("p-value:", p_value)

Kendall's tau: 0.8430560440855659
p-value: 9.032270074512553e-226


In [70]:
sorted_df = df_all_teams.sort_values(by='PrivateScore', ascending=False)
n_cases = sorted_df.head(100)

public_lb = n_cases['PublicScore']
private_lb = n_cases['PrivateScore']
tau, p_value = stats.kendalltau(public_lb, private_lb)

# Print the calculated tau and p-value
print("Kendall's tau:", tau)
print("p-value:", p_value)

Kendall's tau: 0.1532396714549361
p-value: 0.023977519998514583


In [71]:
sorted_df = df_all_teams_submission.sort_values(by='PrivateScore', ascending=False)
n_cases = sorted_df.head(200)

public_lb = n_cases['PublicScore']
private_lb = n_cases['PrivateScore']
tau, p_value = stats.kendalltau(public_lb, private_lb)

# Print the calculated tau and p-value
print("Kendall's tau:", tau)
print("p-value:", p_value)

Kendall's tau: 0.7115254154360408
p-value: 1.293608721140627e-50


In [72]:
sorted_df = df_all_teams_submission.sort_values(by='PrivateScore', ascending=False)

df_all_teams_submission_unique_teams = sorted_df.drop_duplicates(subset='TeamName', inplace=False)
print(len(df_all_teams_submission_unique_teams.TeamName.unique()))
public_lb = df_all_teams_submission_unique_teams['PublicScore']
private_lb = df_all_teams_submission_unique_teams['PrivateScore']
tau, p_value = stats.kendalltau(public_lb, private_lb)

# Print the calculated tau and p-value
print("Kendall's tau:", tau)
print("p-value:", p_value)

365
Kendall's tau: 0.8381381064928418
p-value: 2.8370895200241017e-126


In [40]:
# top submission from all teams, private vs public ranking
indices_to_keep = df_all_teams_submission.groupby('TeamName')['PrivateScore'].nlargest(1).index.get_level_values(1)
df_top_submission_per_team = df_all_teams.iloc[indices_to_keep]

private_sorted_df = df_top_submission_per_team.sort_values(by='PrivateScore', ascending=False)
private_sorted_df = private_sorted_df.reset_index(drop=True)

count = 1
for index, row in private_sorted_df.iterrows():
    private_sorted_df.loc[index, 'TeamRank'] = count
    count +=1
# n_cases = sorted_df.head(15)

indices_to_keep1 = df_all_teams_submission.groupby('TeamName')['PublicScore'].nlargest(1).index.get_level_values(1)
df_top_submission_per_team1 = df_all_teams.iloc[indices_to_keep1]

public_sorted_df = df_top_submission_per_team1.sort_values(by='PublicScore', ascending=False)

# n_cases.head()
print(public_sorted_df.TeamId.count(), private_sorted_df.TeamId.count())

public_lb = public_sorted_df['TeamId']
private_lb = private_sorted_df['TeamId']
tau, p_value = stats.kendalltau(public_lb, private_lb)

# Print the calculated tau and p-value
print("Kendall's tau:", tau)
print("p-value:", p_value)

364 364
Kendall's tau: 0.04988950443495899
p-value: 0.1553420757986034


In [45]:
display(private_sorted_df, public_sorted_df)

,DateSubmitted,SubmissionId,TeamId,UserName,TeamName,IsSelected,PublicScore,PrivateScore
35285,2022-09-22 17:45:15,28201344,8809559,opusen,I need SegFormer,True,0.827166,0.835625
34657,2022-09-22 01:32:27,28188836,8807737,victorsd,Victor Durnov,True,0.841786,0.833936
35075,2022-09-22 13:09:08,28198390,8893081,vladimirsydor,Human Torus Team,True,0.839835,0.832669
35136,2022-09-22 14:27:08,28199042,9015880,rock139,Rock,True,0.839608,0.827177
34970,2022-09-22 10:05:35,28196147,8807628,carnozhao,Carno&Roc,True,0.613091,0.825959
...,...,...,...,...,...,...,...,...
32962,2022-09-19 16:20:49,28164612,9115139,josephmuddle,Joseph Muddle,True,0.221049,0.109414
1755,2022-07-12 23:42:22,27417541,8815063,ubutton,ubutton,True,0.117173,0.084077
25159,2022-09-08 06:12:45,28045408,9125850,del=834dc156a8f21839,[Deleted] c1cc31d2-b73c-496a-a91d-03df453aa2ca,True,0.000000,0.000000
14653,2022-08-22 11:20:56,27888041,9040694,hidetotsuzuki,hideto_tsuzuki,True,0.000000,0.000000


,DateSubmitted,SubmissionId,TeamId,UserName,TeamName,IsSelected,PublicScore,PrivateScore
34657,2022-09-22 01:32:27,28188836,8807737,victorsd,Victor Durnov,True,0.841786,0.833936
34749,2022-09-22 04:09:44,28191185,8993571,qdv206,HCMUT_Alumni,True,0.839835,0.822673
35075,2022-09-22 13:09:08,28198390,8893081,vladimirsydor,Human Torus Team,True,0.839835,0.832669
35136,2022-09-22 14:27:08,28199042,9015880,rock139,Rock,True,0.839608,0.827177
34900,2022-09-22 07:57:42,28194378,8928461,qtakka,Q_takka,True,0.836843,0.825170
...,...,...,...,...,...,...,...,...
2152,2022-07-15 09:10:42,27464296,8856260,riverdeer,Byeonghun,True,0.170309,0.162811
1755,2022-07-12 23:42:22,27417541,8815063,ubutton,ubutton,True,0.117173,0.084077
14653,2022-08-22 11:20:56,27888041,9040694,hidetotsuzuki,hideto_tsuzuki,True,0.000000,0.000000
26601,2022-09-10 07:40:50,28062569,9089687,zhenhuanhuang,zhenhuan huang,True,0.000000,0.000000


In [62]:

# Top 50 teams best submission
indices_to_keep = df_all_teams_submission.groupby('TeamName')['PrivateScore'].nlargest(1).index.get_level_values(1)
df_top_submission_per_team = df_all_teams.iloc[indices_to_keep]

sorted_df = df_top_submission_per_team.sort_values(by='PrivateScore', ascending=False)
n_cases = sorted_df.head(15)

n_cases.head()
print(n_cases.TeamId.count())

public_lb = n_cases['PublicScore']
private_lb = n_cases['PrivateScore']
tau, p_value = stats.kendalltau(public_lb, private_lb)

# Print the calculated tau and p-value
print("Kendall's tau:", tau)
print("p-value:", p_value)

15
Kendall's tau: 0.18095238095238098
p-value: 0.3794503959627968


In [ ]:
n_cases

# Top 50 teams analysis

In [17]:
column_names = ['TeamName', 'dice_score', 'iou_score', 'hd_score', 'hd95_score']
df_all_metrics_top50 = pd.DataFrame(columns=column_names)
df_all_metrics_top50

,TeamName,dice_score,iou_score,hd_score,hd95_score


In [ ]:
for i in range(1,51):
    # metric = "iou_score"
    team = f"team_{i}"
    if i == 1 or i == 2 or i == 3:
        df_metrics = pd.read_csv(INPUT_PATH+f'winning-submissions/{team}/metrics.csv')
    else:
        df_metrics = pd.read_csv(INPUT_PATH+f'winning-submissions/other_top50_teams/{team}/metrics.csv')
    # sorted_df = df_metrics.sort_values(by='PrivateScore', ascending=False)
    dice_mean = df_metrics['dice_score'].mean()
    iou_mean = df_metrics['iou_score'].mean()
    hd_mean = df_metrics['hd_score'].mean()
    hd95_mean = df_metrics['hd95_score'].mean()

    new_row = {'TeamName' : team, 'dice_score' : dice_mean, 'iou_score' : iou_mean, 'hd_score' : hd_mean, 'hd95_score' : hd95_mean}

    df_all_metrics_top50 = df_all_metrics_top50.append(new_row, ignore_index=True)




In [19]:
display(df_all_metrics_top50)

,TeamName,dice_score,iou_score,hd_score,hd95_score
0,team_1,0.835616,0.738462,201.991753,380.928247
1,team_2,0.833929,0.736247,261.016578,392.336002
2,team_3,0.832661,0.733348,268.145931,406.362649
3,team_4,0.824744,0.724394,202.038490,389.344642
4,team_5,0.825953,0.727103,186.205165,363.487386
5,team_6,0.825355,0.727711,212.415925,368.317201
6,team_7,0.825162,0.725917,289.557443,386.677818
7,team_8,0.824804,0.724025,240.503372,399.504098
8,team_9,0.823342,0.724083,178.454873,369.256189
9,team_10,0.822665,0.722829,249.206046,385.919631


In [20]:
# df_all_metrics_top50_sorted_dice = df_all_metrics_top50.sort_values(by='dice_score', ascending=False)
# df_all_metrics_top50_sorted_iou = df_all_metrics_top50.sort_values(by='iou_score', ascending=False)
# df_all_metrics_top50_sorted_hd = df_all_metrics_top50.sort_values(by='hd_score', ascending=False)
# df_all_metrics_top50_sorted_hd95 = df_all_metrics_top50.sort_values(by='hd95_score', ascending=False)

In [21]:
arg1 = df_all_metrics_top50['dice_score']
arg2 = df_all_metrics_top50['iou_score']
tau, p_value = stats.kendalltau(arg1, arg2)

# Print the calculated tau and p-value
print("Kendall's tau:", tau)
print("p-value:", p_value)

Kendall's tau: 0.9232653061224491
p-value: 3.060227213394854e-21


In [22]:
arg1 = df_all_metrics_top50['dice_score']
arg2 = df_all_metrics_top50['hd_score']
tau, p_value = stats.kendalltau(arg1, arg2)

# Print the calculated tau and p-value
print("Kendall's tau:", tau)
print("p-value:", p_value)

Kendall's tau: -0.11673469387755102
p-value: 0.23162840695170472


In [23]:
arg1 = df_all_metrics_top50['dice_score']
arg2 = df_all_metrics_top50['hd95_score']
tau, p_value = stats.kendalltau(arg1, arg2)

# Print the calculated tau and p-value
print("Kendall's tau:", tau)
print("p-value:", p_value)

Kendall's tau: -0.43510204081632653
p-value: 8.254705896707545e-06
